# M1M3 Hardpoint Measured Forces analysis

## Setup notebook

In [ ]:
# Define which day_obs to analyze
DAY_OBS : int = 20251204

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import matplotlib.pyplot as plt
import pandas as pd

from astropy.time import Time, TimeDelta
from pathlib import Path
from tqdm import tqdm

from lsst.summit.utils.m1m3 import inertia_compensation_system as ics
from lsst.summit.utils.m1m3.plots import plot_ics

from lsst.summit.utils.dateTime import getDayObsEndTime, getDayObsStartTime
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import (
    getCommandsDuringEvent,
    TMAEvent,
    TMAEventMaker,
    TMAState,
)

import BLOCK_T227_utils as block

In [ ]:
plot_path = Path("./plots")
plot_path.mkdir(exist_ok=True, parents=True)

tma_event_maker = TMAEventMaker()
efd_client = makeEfdClient()

logger = logging.getLogger("m1m3_analysis")
logger.setLevel(logging.CRITICAL)

## Query slew events

In [ ]:
tma_events = tma_event_maker.getEvents(DAY_OBS)
track_events = [e for e in tma_events if e.type == TMAState.TRACKING]
slew_events = [e for e in tma_events if e.type == TMAState.SLEWING]

print(
    f"Found {len(tma_events)} TMA Events on day_obs={DAY_OBS}\n"
    f"  Number of tracking events: {len(track_events)}\n"
    f"  Number of slewing events: {len(slew_events)}\n"
)

## Analyze all the slews

In [ ]:
mix_max_forces = []

# Part of the code below is implemented somewhere
# I cannot find it, so let's repeat. That's fine for now.
min_forces_columns = [f"measuredForceMin{i}" for i in range(6)]
max_forces_columns = [f"measuredForceMax{i}" for i in range(6)]

# Build our new data frame
for event in tqdm(slew_events):
    m1m3_data = ics.M1M3ICSAnalysis(event, tma_event_maker.client, log=logger)
    m1m3_data.log.setLevel("ERROR") 
    
    min_forces = m1m3_data.stats[min_forces_columns].min()
    max_forces = m1m3_data.stats[max_forces_columns].max()
    
    mix_max_forces.append([min_forces, max_forces])

# Convert into a data frame for plotting
df = pd.DataFrame(data=mix_max_forces, columns=["min_forces", "max_forces"])

In [ ]:
operational_limit = 450
fatigue_limit = 900
valid_data = df.dropna()
print(f"Valid slews: {valid_data.index.size}")

red_range_mask = (df.min_forces.abs() >= fatigue_limit) + (df.max_forces.abs() >= fatigue_limit)
red_range_df = df[red_range_mask]
print(f"Number of slews above fatigue limit (900N) = {red_range_df.index.size} ({red_range_df.index.size/valid_data.index.size:.2%})")

yellow_range_mask = ((df.min_forces.abs() >= operational_limit) * (df.min_forces.abs() <= fatigue_limit)) + ((df.max_forces.abs() >= operational_limit) * (df.max_forces.abs() <= fatigue_limit))
yellow_range_df = df[yellow_range_mask]
print(f"Number of slews above operational limit (450N) and below fatigue limit (900N) = {yellow_range_df.index.size} ({yellow_range_df.index.size/valid_data.index.size:.2%})")

## Plot and save figure

In [ ]:
block.plot_histogram_hp_minmax_forces(df, DAY_OBS)